<a href="https://colab.research.google.com/github/ykitaguchi77/GAN/blob/master/ChatGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [97]:
from google.colab import drive
drive.mount("/content/drive")

!pip install openai --q
!pip install --upgrade deepl --q
import openai
import deepl
import textwrap
import pandas as pd

with open("/content/drive/MyDrive/Deep_learning/api.txt") as file:
    #text = file.read()
    i=1
    key = []
    while True:
        include_break_line = file.readline() #改行が含まれた行
        line = include_break_line.rstrip() #改行を取り除く
        if line: #keyの読み込み
            #print(f'{i}行目：{line}')
            key.append(line)
            i += 1
        else:
            break

# APIキーの準備
#ngrok_aceess_token = key[5]
openai.api_key = key[3]
deepl_auth_key = key[1]

translator = deepl.Translator(deepl_auth_key)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [98]:
#@title #**ChatGPT configulation**
question_translate_English= True #@param {type:"boolean"}
answer_Japanese= False #@param {type:"boolean"}
reset= True #@param {type:"boolean"}
show_all = True #@param {type:"boolean"}
arrangement = True #@param {type:"boolean"}



if "text_input" not in globals():
    text_input = []
if reset == True:
    text_input = []


def wrap(sentence):
    wrap_list =  textwrap.wrap(sentence, 120)
    output = '\n'.join(wrap_list)
    return(output)


def interference(question):
    question = question.replace('\n', '') #前後の空白と改行を削除

    if question_translate_English == True:
        question_translated = translator.translate_text(question, target_lang="EN-US").text
        question = question_translated
    else:
        question = question

    prompt = "Q:" + question +"\nA:"

    text_input.append(question+"\nA:")

    prompt =  ",".join(map(str, text_input)) # aaa,bbb,100,200,300 

    response = openai.Completion.create(
      model="text-davinci-003",
      prompt=prompt,
      temperature=0.5,
      max_tokens= 1024,
      #top_p=1,
      frequency_penalty=0.0,
      presence_penalty=0.0,
      stop=None)

    answer = response['choices'][0]['text']
    text_input.append(answer)

    if answer_Japanese == True:
        answer = translator.translate_text(response['choices'][0]['text'], target_lang="JA").text
    else:
        answer = answer

    output(text_input, show_all=show_all, arrangement=arrangement)
    # print(f"{prompt}")
    # print(f"{wrap(answer)} \n")
    #print(text_input)


def output(text_input, show_all = True, arrangement = False):
    if arrangement == True:
        if show_all == False:
            print(f"Q: {wrap(text_input[-2]).rstrip('A:')} \n")
            print(f"A: {wrap(text_input[-1])} \n")

        elif show_all == True:
            for i, text in enumerate(text_input):
                if i % 2 == 0:
                    print(f"Q: {wrap(text).rstrip('A:')} \n")
                else:
                    print(f"A: {wrap(text)} \n")

    elif arrangement == False:
        if show_all == False:
            print(f"Q: {text_input[-2].rstrip('A:')} \n")
            print(f"{text_input[-1]} \n")

        elif show_all == True:
            for i, text in enumerate(text_input):
                if i % 2 == 0:
                    print(f"Q: {text.rstrip('A:')} \n")
                else:
                    print(f"{text} \n")




In [ ]:
#@title #**Question & Answer list**

Leave_first = 2 #@param {type:"slider", min:0, max:10, step:2}

Leave_last = 0 #@param {type:"slider", min:0, max:10, step:2}

tmp = []

if Leave_first == 0:
    pass
else:
    tmp.extend(text_input[0:Leave_first])

if Leave_last == 0:
    pass
else:
    tmp.extend(text_input[-1*Leave_last:])

text_input = tmp
output(text_input)


In [95]:
Delete_last_two= True #@param {type:"boolean"}
if Delete_last_two == True:
    text_input = text_input[:-2]

output(text_input)

Q: 次の文を、誤解のないように言い換えて下さい A challenge in diagnostic AI to assist physicians is the possibility that the cost of development may not match the expected benefits.
 

 The potential challenge of developing diagnostic AI to assist physicians is that the cost may not be commensurate with the anticipated rewards. 



#**Interference**

In [96]:
question = '''

次の文を、単語を使って言い換えて下さい。かつ誤解のないようにして下さい
The potential challenge of developing diagnostic AI to assist physicians is that the cost may not be commensurate with
the anticipated rewards. 

'''
interference(question)

Q: 次の文を、誤解のないように言い換えて下さい A challenge in diagnostic AI to assist physicians is the possibility that the cost of development
may not match the expected benefits.  

A:  The potential challenge of developing diagnostic AI to assist physicians is that the cost may not be commensurate with
the anticipated rewards. 

Q: Paraphrase the following sentence using the words: The potential challenge of developing diagnostic AI to assist
physicians is that the cost may not be commensurate withthe anticipated cost. The potential challenge of developing
diagnostic AI to assist physicians is that the cost may not be commensurate withthe anticipated rewards.  

A: The potential challenge of developing diagnostic AI to assist physicians is that the cost may not be proportional to the
expected benefits. 



In [ ]:
output(text_input, show_all = True, arrangement = False)

Q: I need you to act as a research assistant for a scientific article I am writing. I will provide you with specific questions and information, and I expect you to find relevant and credible sources to support my research. Your responses should include a summary of the main points from the sources, along with the full citation in the desired format (APA, MLA, etc.). Additionally, you should flag any potential issues or limitations in the sources you provide. The title of the article is: Real-time smartphone identification of corneal diseases displayed on computer screen using embedded deep-learning model. Please show me the bullets which I need to contain in the introduction. The number of bullets should be 4 to 6.
 

 
• Introduction to corneal diseases and their diagnosis 
• Overview of existing corneal disease diagnosis methods 
• Description of deep learning models for diagnosis of corneal diseases
• Advantages of using real-time smartphone identification of corneal diseases
• Limi

# **Save and load data**

In [ ]:
save_dir = "/content/drive/MyDrive/Deep_learning/chatGPT"
file_name = "corneAI"


# save_data
import pickle
f = open(f'{save_dir}/{file_name}.txt', 'wb')
list_row = text_input
pickle.dump(list_row, f)

In [ ]:
# load data
f = open(f'{save_dir}/{file_name}.txt', 'rb')
text_input = pickle.load(f)